In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git


import sklearn
import sklearn.feature_extraction.text

#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

In [2]:
df = pandas.read_csv('t\df_70.csv', index_col=0)

In [3]:
#Apply our functions
df['tokenized_text'] = df['AB'].apply(lambda x: lucem_illud_2020.word_tokenize(x))
df['normalized_tokens'] = df['tokenized_text'].apply(lambda x: lucem_illud_2020.normalizeTokens(x))

In [4]:
ngTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, 
                                                                 max_features=1000, min_df=3, 
                                                                 stop_words='english', 
                                                                 norm='l2')
groupsTFVects = ngTFVectorizer.fit_transform(df['AB'])

In [5]:
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]

df['reduced_tokens'] = df['normalized_tokens'].apply(lambda x: dropMissing(x, ngTFVectorizer.vocabulary_.keys()))

In [6]:
dictionary = gensim.corpora.Dictionary(df['reduced_tokens'])

In [7]:
## create a list of tuples containing each token and its count 
## use the first half and save the remainder for testing 
corpus = [dictionary.doc2bow(text) for text in df['reduced_tokens']]

In [8]:
# serialize the corpus as a file and load it 
gensim.corpora.MmCorpus.serialize('abstract.mm', corpus)
abmm = gensim.corpora.MmCorpus('abstract.mm')

In [9]:
# correctly formatted corpus that can be used for topic modeling and induction 
senlda = gensim.models.ldamodel.LdaModel(corpus=abmm, id2word=dictionary, num_topics=20, alpha='auto', eta='auto')

In [10]:
# sen1Bow = dictionary.doc2bow(df['reduced_tokens'][0])
# sen1lda = senlda[sen1Bow]
# print("The topics of the text: {}".format(df['TI'][0]))
# print("are: {}".format(sen1lda))

In [11]:
ldaDF = pandas.DataFrame({
        'titles' : df['TI'],
        'topics' : [senlda[dictionary.doc2bow(l)] for l in df['reduced_tokens']]
    })

In [12]:
topicsProbDict = {i : [0] * len(ldaDF) for i in range(senlda.num_topics)}

for index, topicTuples in enumerate(ldaDF['topics']):
    for topicNum, prob in topicTuples:
        topicsProbDict[topicNum][index] = prob

for topicNum in range(senlda.num_topics):
    ldaDF['topic_{}'.format(topicNum)] = topicsProbDict[topicNum]

In [13]:
K = senlda.num_topics  # N documents, K topics
topic_labels = ['Topic #{}'.format(k) for k in range(K)]

topicsDict = {}
for topicNum in range(senlda.num_topics):
    topicWords = [w for w, p in senlda.show_topic(topicNum)]
    topicsDict['Topic_{}'.format(topicNum)] = topicWords

wordRanksDF = pandas.DataFrame(topicsDict)
wordRanksDF

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Topic_10,Topic_11,Topic_12,Topic_13,Topic_14,Topic_15,Topic_16,Topic_17,Topic_18,Topic_19
0,concept,variable,health,change,health,performance,affect,behavior,group,health,fertility,study,determinant,patient,environment,study,class,female,group,group
1,sex,determinant,family,factor,patient,use,difference,group,life,determinant,perspective,treatment,effect,treatment,behavior,determinant,experience,male,child,health
2,self,human,determinant,marital,effect,therapy,examine,sex,variable,group,family,information,man,determinant,factor,family,temporal,infant,age,year
3,distance,analysis,study,determinant,determinant,female,research,difference,old,life,suicide,activity,study,complete,variable,use,variable,day,control,study
4,group,influence,increase,year,study,self,control,drug,determinant,factor,marital,vaccination,result,female,determinant,health,attitude,test,infant,individual
5,determinant,fertility,consumer,life,mental,achievement,factor,research,person,patient,size,result,life,factor,sex,status,study,behavior,reaction,medical
6,male,political,mental,approach,survey,study,health,use,age,planning,difference,practice,variable,student,report,research,function,interaction,suggest,drug
7,study,behavior,group,follow,work,experience,child,individual,different,study,equation,reaction,role,economic,patient,preventive,child,determinant,determinant,experience
8,child,measure,process,sex,area,old,problem,high,experience,behavior,model,health,difference,general,health,work,sex,mother,behavior,patient
9,factor,economic,self,increase,major,political,study,finding,compliance,experience,drug,vary,economic,rate,sexual,report,relationship,factor,response,member


In [21]:
import pyLDAvis.gensim
lda_model = senlda
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.008164 -0.017490       1        1  14.185807
1     -0.044309  0.076384       2        1   9.493976
14    -0.073171  0.061315       3        1   7.725306
15    -0.010779  0.047866       4        1   7.217715
4      0.057940  0.045920       5        1   6.525156
9      0.025340 -0.073150       6        1   6.505334
12    -0.025366  0.038730       7        1   5.911579
6      0.010608  0.004676       8        1   5.576561
17     0.027569 -0.077527       9        1   5.418237
0     -0.094693 -0.023604      10        1   4.742652
13     0.152034 -0.027036      11        1   4.364347
19     0.042273 -0.047009      12        1   3.249477
18    -0.077744 -0.156650      13        1   3.091956
11     0.009277  0.011650      14        1   2.877851
16    -0.110384  0.078794      15        1   2.826072
5      0.090577  0.060020      16        1   2.649887
3      0.018823 -0.005544      17        1   2.263974
8     -0.022199 -0.050342      18        1   2.077190
7     -0.059575 -0.004508      19        1   1.940876
10     0.075614  0.057505      20        1   1.356053, topic_info=    Category       Freq       Term      Total  loglift  logprob
148  Default  56.000000    patient  56.000000  30.0000  30.0000
13   Default  58.000000      group  58.000000  29.0000  29.0000
71   Default  41.000000      child  41.000000  28.0000  28.0000
112  Default  25.000000      class  25.000000  27.0000  27.0000
385  Default  22.000000  fertility  22.000000  26.0000  26.0000
..       ...        ...        ...        ...      ...      ...
357  Topic20   0.835976     female  31.468763   0.6724  -4.5418
107  Topic20   0.815467     report  27.919928   0.7673  -4.5666
45   Topic20   0.696543     effect  46.207363   0.1058  -4.7242
95   Topic20   0.662370    finding  22.883564   0.7582  -4.7745
212  Topic20   0.659694      paper  17.571178   1.0184  -4.7786

[1080 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
375       1  0.156457  ability
375       2  0.156457  ability
375       3  0.156457  ability
375       4  0.156457  ability
375      16  0.156457  ability
...     ...       ...      ...
110      13  0.068293     year
110      14  0.034146     year
110      16  0.034146     year
110      17  0.068293     year
110      18  0.034146     year

[2795 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 15, 16, 5, 10, 13, 7, 18, 1, 14, 20, 19, 12, 17, 6, 4, 9, 8, 11])